# 因果语言模型训练

In [5]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling,  TrainingArguments, Trainer, BloomForCausalLM

In [6]:
ds = Dataset.load_from_disk("../data/wiki_cn_filtered")
print(ds, "\n", ds[0])

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
}) 
 {'source': 'wikipedia.zh2307', 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}


In [7]:
tokenizer = AutoTokenizer.from_pretrained("../Langboat/bloom-389m-zh")

def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]] # 添加结束标记
    return tokenizer(contents, max_length=384, truncation=True)

In [8]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [9]:
from torch.utils.data import DataLoader
# collate_fn用于指定如何将多个样本数据合并成一个batch，mlm指定是否使用掩码语言模型：Masked Language Modeling
# mlm=False表示不进行掩码操作，适用于因果语言模型
dl = DataLoader(tokenized_ds, batch_size=2, 
                collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False)) 

In [10]:
# enumerate:将对象转换为一个枚举对象并返回：（index，batch）元组，index是batch的索引
# next:返回枚举对象的下一个值，第一次调用表示反返回第一个批次的数据
next(enumerate(dl))

(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [11]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [12]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

In [13]:
model = AutoModelForCausalLM.from_pretrained("../Langboat/bloom-389m-zh")

In [14]:
args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16, # 梯度累加，结合per_device_train_batch_size=2，表示累加32个batch
    logging_steps=50,
    num_train_epochs=1,
    fp16=True, # fp16精度训练，加速训练
)

In [15]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

C:\Users\柒\AppData\Local\Temp\ipykernel_32732\3774039206.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
d:\miniconda3\envs\llm\lib\site-packages\accelerate\accelerator.py:449: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [16]:
trainer.train()

  0%|          | 0/312 [00:00<?, ?it/s]

{'loss': 3.9978, 'grad_norm': 706.550537109375, 'learning_rate': 4.3910256410256415e-05, 'epoch': 0.16}


KeyboardInterrupt: 

In [17]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安市大唐芙蓉园，西安铁路铁路枢纽西安市火车站的综合性博物馆。馆藏有中国古代珍贵玉器和珍贵雕像。展藏面积达三十多吨。为国内唯一一座拥有国内重点文物保护单位之博物馆。馆藏玉器和雕像均以珍贵方式收藏，其中出土的一尊秦始皇大型雕像，出土的秦始皇小型雕像，出土的一尊中国古朴、朴实的名人雕像等珍贵雕像是展馆的精品。\n该展馆为二层，五楼相连，楼檐下为歇山式檐"}]

In [20]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常迅猛，许多的游戏玩家们在游戏里得到了一些收益，也得到了他们想要的东西，比如，他们想拥有的是比他们自己的游戏更高的利润。由于《尤达求生存》、《全境封锁》、《堡垒之夜》、《战锤2》、《战争》、《暴躁怪物》、《暴躁怪物2》、《生化危机5》、《生化危机》（生化危机 5是所有系列第一部的续集）是第一款由索尼和TNT合作制作的动作类SAP游戏，而《战争》、《暴'}]